In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("ecommerce")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
# in single spark driver/note book/spark application,
# there can be many spark sessions, and 
# only one spark context
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()
# spark core operations, like rdd, partitions, actions etc
# spark session shall use catalyst engine, which will use spark context for low level
# code execution
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 08:09:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/19 08:09:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/19 08:09:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/19 08:09:24 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/19 08:09:24 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/05/19 08:09:24 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [3]:
ecommerceDf = spark.read.format("csv")\
                .option("header", True)\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/ecommerce/data.csv")
ecommerceDf.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [4]:

from pyspark.sql.types import DoubleType,StructType, StringType, IntegerType, DateType
schema = StructType() \
         .add("InvoiceNo", StringType(), True) \
         .add("StockCode", StringType(), True) \
         .add("Description", StringType(), True) \
         .add("Quantity", IntegerType(), True) \
         .add("InvoiceDate", DateType(), True) \
         .add("UnitPrice", DoubleType(), True) \
         .add("CustomerID", IntegerType(), True) \
         .add("Country", StringType(), True)


ecomDf = spark.read.format("csv") \
                .option("header", False) \
                .schema(schema) \
                .option("dateFormat", "MM/dd/yyyy HH:mm")\
                .load("hdfs://localhost:9000/ecommerce/data.csv")

ecomDf.show(2)

22/05/19 08:09:44 ERROR Executor: Exception in task 0.0 in stage 3.0 (TID 3)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '12/1/2010 8:26' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601DateFormatter.parse(DateFormatter.scala:57)
	at org.apache.spark.sql.catalyst.csv

Py4JJavaError: An error occurred while calling o44.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (10.0.0.164 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '12/1/2010 8:26' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartia